# Análisis de la distribución de la popularidad (número de reseñas en el primer mes) #

El objetivo de este notebook es analizar la distribución del número de reseñas de los juegos en Steam

In [1]:
import pandas as pd
import json
import gzip
import os
import matplotlib.pyplot as plt
import plotly.express as px
from pathlib import Path

#### Creación del dataset:
Es necesario leer el json y crear un dataset a patir de la lista asociada a la clave `data`.

In [ ]:
base_dir = Path().resolve().parents[1]
data_dir = base_dir / "data"

# No usa variable de entorno pues la ruta será el archivo final con todos los juegos,
# no partes individuales de cada identificador
ruta = Path(data_dir / f"info_steam_games_3.json.gz")

if not ruta.exists():
    raise FileNotFoundError(f"No se encuentra la ruta: {ruta}")

with gzip.open(data_dir / "info_steam_games_3.json.gz", "rt", encoding="UTF-8") as f:
    data = json.load(f)
    
df = pd.DataFrame(data['data'])

df

#### Modificación del dataframe:
Creación de columnas `free` que sirve para separar las distribuciones de los juegos gratuitos de los no gratuitos

In [3]:
df["name"] = df["appdetails"].apply(lambda x : x.get("name") if isinstance(x,dict) else None)
df["free"] = df["appdetails"].apply(lambda x: True if x.get("price_overview").get("initial") == 0 else False)
df.drop(columns=["appdetails"], inplace=True)

Creación de las columnas `recomendaciones_positivas` y `recomendaciones_negativas`.

In [4]:
df["recomendaciones_positivas"] = df["appreviewhistogram"].apply(lambda x: x.get("rollups").get("recommendations_up") if isinstance(x, dict) & isinstance(x.get("rollups"), dict) else None)
df["recomendaciones_negativas"] = df["appreviewhistogram"].apply(lambda x: x.get("rollups").get("recommendations_down") if isinstance(x, dict) & isinstance(x.get("rollups"), dict) else None)
df.dropna(inplace=True)

Se comprueba que no hay nulos en ninguna de las 2 columnas creadas

In [ ]:
print(f"Nulos en columna recomendaciones positivas: {df["recomendaciones_negativas"].isna().sum()}, Nulos en columna recomendaciones negativas: {df["recomendaciones_positivas"].isna().sum()}")

Por último se ordena el dataset por el número de reseñas totales

In [ ]:
df["total"] = df["recomendaciones_positivas"] + df["recomendaciones_negativas"]

df.sort_values(by="total",ascending=False, inplace=True)

df.head(n=20)

In [ ]:
plt.hist(x=df["total"], bins=100)
plt.yscale("log")
plt.xlabel("Numero de reseñas")
plt.ylabel("Número de juegos (log)")
plt.title("Distribución de popularidad (número de reseñas en el primer mes)")
plt.show()

In [ ]:
fig = px.histogram(df, x = "total",title="Distribución de popularidad (número de reseñas en el primer mes)",
                   labels={"x":"Número de reseñas", "y":"Número de juegos (log)"},nbins= 100,color = "free", opacity= 0.9,facet_col="free")
fig.update_yaxes(type="log")

fig.show()

In [ ]:
print(f"Número de juegos con menos de 10 reseñas en el primer mes: {df[df["total"] < 50].shape[0]}")

Conclusiones:

Se aprecia que la distribución era la que esperábamos, una de cola larga con gran concentración de juegos alrededorde números de reseñas bajos y solo unos pocos con muchas reseñas. Además parece que la distribución se mantiene en las distintas categorías (los juegos gratuitos presentan la misma distribución que los no gratuitos).

Se puede comprobar si la distribución se mantiene a lo largo de distintas categorías diferentes. -> COMPLETAR